# Module 11: Phylogenetics

## Overview

Phylogenetics is the study of evolutionary relationships among biological entities – often species, individuals or genes (which may be referred to as taxa). The major elements of phylogenetics are summarised in figure below.

![Intro](./images/phylo1.png)

*Taken from: https://www.ebi.ac.uk/training/online/courses/introduction-to-phylogenetics/what-is-phylogenetics/*


Phylogenetic trees based on whole genome data tell us about the relationships of bacterial isolates to each other on a very fine scale. When we combine that high-resolution information about the evolutionary relationships of isolates with geographical data it can inform our understanding of the current distribution of the pathogen and allow us to infer the epidemiological processes that have acted on the bacteria over time. The simplest example of this would be if a phylogeny showed that a pathogen was geographically constrained (e.g. isolates from the same region always cluster together). This might indicate that the pathogen is not rapidly spreading. Whereas a pathogen with a phylogeny that shows isolates from distant regions are likely to be related to isolates from nearby, the interpretation is that the pathogen is likely to be spread across regional borders. Geographical referencing of genomic data can also be combined with temporal information to study the movement of pathogens in space and time. This is most useful when done in real time and thus can be useful for outbreak detection and monitoring.


This module is divided into three parts where we will address:

1. Construction of phylogenetic trees
2. Identification of recombination regions
3. Clustering using popPUNK.

### Instalall condacolab

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
!conda config --add channels conda-forge
!conda config --add channels defaults
!conda config --add channels bioconda

### Install software

In [ ]:
# Install FastTree
!conda install bioconda::fasttree

In [ ]:
# Install snp-sites
!conda install snp-sites

In [ ]:
# Istall Gubbins
!conda install gubbins

In [ ]:
# Install PopPunk
!conda install poppunk

### Descargar datos

In [ ]:
!wget

### Extract the .tar.gz file 

In [ ]:
!tar xvf Module_13.tar.gz

## Part 1: Creating a phylogenetic tree using FastTree

### Step 1: Make a SNP-only alignment using snp-sites

Creating a phylogeny from whole genome sequences can be a very time consuming and computationally intensive process. We can speed this up by only using the variable sites (SNPs). However, we need to be aware that only including variable sites can affect the evolutionary rate estimates made by phylogenetics software - therefore, we need to account for the sites we remove in our analysis.

We will use snp-sites to do this. You can view the options for snp-sites using the command:

In [ ]:
%cd Module_13

In [ ]:
# Run snp-sites
!snp-sites -h

First, remove all the invariant sites and create a SNP-only multiple sequence alignment. We will use output from snippy runs described in the previous page. Run the command: 

In [ ]:
!snp-sites -o clean.full.SNPs.aln clean.full.aln

An explanation of this command is as follows:

`snp-sites`: is the tool/program

`-o clean.full.SNPs.aln`: specifies the output file

`clean.full.aln`: specifies the input file - which is an output of snippy 

We can see how many invariant sites were removed (and what proportion of A, T, G, C they were) using:

In [ ]:
!snp-sites -C clean.full.aln

You will obtain the following output:

### Step 2: Creating a phylogenetic tree from the SNPs using FastTree

You can view options for fasttree as follows:

In [ ]:
!FastTree -h

We will generate a maximum likelihood phylogenetic tree using this command:

In [ ]:
!FastTree -nt -gtr clean.full.SNPs.aln > clean.full.SNPs.aln.tree

An explanation of this command is as follows:

`FastTree`:  is the tool/program

`-nt`: specifies that the input alignment is nucleotide  

`gtr`:  specifies the evolutionary model 

`clean.full.SNPs.aln`: input alignment

`clean.full.SNPs.aln.tree`: specifies the output tree name

Let’s explore output of our previous command using the command:

In [ ]:
!ls -lh clean.full.*

Our maximum likelihood tree is labelled clean.full.SNPs.aln.tree. We can visualise this using figtree or iTOL

### Step 3: Visualization of a Tree

Download the file `clean.full.SNPs.aln.tree` to your computer by clicking on the "File" icon on the left side of Colab and go to [https://itol.embl.de/](https://itol.embl.de/).

In the upper left corner, select the "Upload" option as shown below:

![iTOL](images/itol3.png)

Then, click on the "Browse..." button, select your .tree file, and click "Upload":

![iTOL](images/itol4.png)

Finally, you will obtain the following tree:

![iTOL](images/itol.png)


Or using the ggtree package in R

```
library(ggtree)

mltree <- midpoint.root(read.tree("clean.full.SNPs.aln.treef")) 

ggtree(mltree) + # plot basic tree

geom_tiplab(size=3) + # add tip labels

geom_treescale() + # add scale bar

xlim(0, 0.0005) # set limits so the plot fits nicely on the screen

```


### Step 3: Interpreting a phylogenetic tree 

We need remember that the tree we have created comes from an alignment of SNPs only. Remember that the longer the branch of a strain, the more mutations it has. The strain from sample ERR2667737 and the reference strain are similar to each other and have some mutations that are also found in the strain from sample ERR2667694. The strains from samples ERR2667707 and ERR2667708 share mutations and present different mutations compared to the other strains.

___

## Part 2: Identifying recombination using Gubbins

Many bacteria participate in high rates of homologous recombination. This means that they donate and receive segments of DNA from each other. In the context of a phylogenetic tree, in which we are comparing similar and dissimilar regions to determine relatedness of isolates this can prove problematic resulting in branch lengths which reflect recombinations rather than divergence. This is very important for *Streptococcus pneumoniae* which is naturally competent, meaning it can readily take up DNA. 

[Gubbins](https://github.com/nickjcroucher/gubbins/blob/master/docs/gubbins_manual.md) (Genealogies Unbiased By recomBinations In Nucleotide Sequences) is an algorithm that iteratively identifies loci containing elevated densities of base substitutions while concurrently constructing a phylogeny based on the putative point mutations outside of these regions. Simulations demonstrate the algorithm generates highly accurate reconstructions under realistic models of short-term diversification of sequences through both point mutation and recombination, and can be run on alignments of many hundreds of bacterial genome sequences. It is therefore not appropriate for looking at recombination across species-wide diversity - this can be done gene-by-gene using software such as fastGEAR. Instead, it works on samples of limited diversity, sharing a recent common ancestor - a strain or lineage.

The required input file for Gubbins is a whole genome FASTA alignment. Each sequence should have a unique identifier, and special characters should be avoided. The sequences should only use the characters ACGT (DNA bases),N (unknown base) or - (alignment gap). If a starting tree is to be included, then this should be a Newick format.The alignment is most easily generated through mapping sequences against a reference sequence. This can be achieved with the popular mapping software Snippy. 

You can view Gubbins commands as follows:

In [ ]:
!run_gubbins.py -h

We will run the gubbins tool on a full genome alignment and not SNPs alignment. Now let’s run this command:

In [ ]:
!run_gubbins.py --mar -p output clean.full.aln

An explanation of this command is as follows:

`run_gubbins.py`: is the tool/program

`--mar`: This option tells Gubbins to use a Markov chain Monte Carlo (MCMC) algorithm for inferring the evolutionary history of the genomes.

`-p output`: This option specifies the name of the output directory where the results of the Gubbins analysis will be saved. As /data dir is the mount point of $PWD of your host due to -v, you will see the files in the $PWD of your host

`clean.full.aln`: input file

This command can take a few minutes to run.

Let's look at what gubbins has done using the command (`ls -l output.*`)

An explanation of output files:

`output.branch_base_reconstruction.embl`:  Base substitution reconstruction in EMBL format

`output.recombination_predictions.embl`: Recombination predictions in EMBL file format.

`salida.predicciones_recombinación.gff`: Recombination predictions in GFF format

`salida.sitios_polimorfos_filtrados.phylip`: Phylip format alignment of filtered polymorphic sites used to generate the phylogeny in the final iteration

`output.final_tree.tre`: this file contains the final phylogeny in Newick format; branch lengths are in point mutations

`output.node_labelled.final_tree.tre`: final phylogenetic tree in Newick format but with internal node labels; branch lengths are in point mutations

`output.log`: log file specifying the software used at each step of the analysis, with accompanying citations

`output.per_branch_statistics.csv`:  per branch reporting of the base substitutions inside and outside recombination events

`output.summary_of_snp_distribution.vcf`:  VCF file summarising the distribution of point mutations

You can explore these output files using `head`command. For example "output.recombination_predictions.gff" is a gff file that contains a record of each recombination block identified, how many SNPs it contains, and what samples are affected.


EJEMPLO

**output.final_tree.tre** is a phylogeny without recombination regions. Read the filtered gubbins tree and plot it with ggtree or visualize it in Figtree or iTOL as explained previously.

You will have the output:

![iTOL](images/itol2.png)



Or using the ggtree package in R

```
!gubbins.tree <- midpoint.root(read.tree("gubbins.final_tree.tre")) 

ggtree(gubbins.tree) + # plot basic tree

geom_tiplab(size=3) + # add tip labels

geom_treescale() + # add a scale bar

xlim(0, 300) # change the sizing of the plot so it fits nicely on the screen 

```

We can also visualise the recombination blocks using a web tool called phandango. Using your browser, navigate to:  https://jameshadfield.github.io/phandango/#/

You will need the following files (drag and drop):

1. output.final_tree.tre
2. output.recombination_predictions.gff
3. reference.gff (output from Prokka of Reference_sequence_GPSC1.fa)

Phandango should automatically display blocks of recombination in red (ancestral) and blue (specific to a sample)

![rec](images/phylocanvas.png)


___

## Part 3: Clustering using popPUNK

[PopPUNK](https://poppunk.readthedocs.io/en/latest/index.html) is a tool for clustering genomes. We refer to the clusters as variable-length-k-mer clusters, or VLKCs. Biologically, these clusters typically represent distinct strains. We refer to subclusters of strains as lineages.

The figure below shows an overview of how to run popPUNK

![punk](./images/poppunk_flowchart.png)

*Taken from: https://poppunk.readthedocs.io/en/latest/overview.html*

### Database

As we are working with *Streptococcus pneumoniae*, we will use the GPS [reference database](https://gps-project.cog.sanger.ac.uk/GPS_v9.tar.gz) and the [GPS designation](https://gps-project.cog.sanger.ac.uk/GPS_v9_external_clusters.csv) that we will use to group our genome. You can also access reference genomes from other bacterial species from this database. If a species is not included in this database, it is recommended that you create your own database.

The GPS reference genome database for *Streptococcus pneumoniae* is saved in your directory as (GPS_v9) and the GPS designation as (GPS_v9_external_clusters.csv).


In [ ]:
# Unzip the database
!tar xvf GPS_v9.tar.gz

### Text file with details of your samples

You need a file that lists the names of your samples and the paths to your sequence data. In this case, the file is: `poppunk_input.tsv`.

You can check it with the following command:

In [ ]:
!cat poppunk_input.tsv

This text file contains the sample names and their sequence data. It has no header, is tab separated, and contains the sample name in the first column. Subsequent columns may contain paths to either assembled or raw read data (the type will automatically be inferred by checking for the presence of quality scores). 

### Clustering your genomes

The command to cluster your genomes is as follows:

In [ ]:
!poppunk_assign --db GPS_v9 --query poppunk_input.tsv --output poppunk_clusters --external-clustering GPS_v9_external_clusters.csv 

An explanation of this command is as follows:

`poppunk_assign`: is the tool/program/script

`--db GPS_v9`: specifies the database

`--query /data/poppunk_input.tsv`: receives input from the list.txt

`--output poppunk_clusters`: specifies the output file 

`--external-clustering GPS_v9_external_clusters.csv`:  directory containing the GPS clusters for the references



Upon completion, a new folder "poppunk_clusters" will be generated. Navigate to this folder and explore its content. 

The output files: 

**poppunk_clusters_clusters.csv**: popPUNK clusters with dataset specific nomenclature

**poppunk_clusters_clusters_externos.csv**: GPSC v9 scheme designations

We can explore "poppunk_clusters_clusters.csv"

>Note: If a strain has already been assigned a cluster, please rename to run popPUNK (this is to avoid crushing the tool). Novel Clusters are assigned NA in the _external_clusters.csv as they have not been defined in the v6 dataset used to designate the GPSCs. Please email: globalpneumoseq@gmail.com to have novel clusters added to the database and a GPSC cluster name assigned after you have checked for low level contamination which may contribute to biased accessory distances.

This popPUNK module was developed from the following webpages:

https://poppunk.readthedocs.io/en/latest/query_assignment.html


https://www.pneumogen.net/gps/training_command_line.html